In [1]:
from deltalake import DeltaTable
from dotenv import load_dotenv

load_dotenv()

import duckdb
import polars as pl
import polars_st as st
import deltalake as dl
import pyarrow as pa
from src.settings import settings
from datetime import datetime
from zoneinfo import ZoneInfo
from httpx import Client
from fake_useragent import UserAgent

In [2]:
%load_ext sql
conn = duckdb.connect(settings.DUCKDB_DATABASE)
conn.sql("SET search_path TO ae_de_play;")
conn.install_extension('spatial')
conn.load_extension('spatial')
conn.install_extension('delta')
conn.load_extension('delta')

/app/.venv/lib/python3.12/site-packages/sql/parse.py:339: SyntaxWarning: invalid escape sequence '\:'
  Given a query, replaces all occurrences of ':variable' with '\:variable' and
/app/.venv/lib/python3.12/site-packages/sql/parse.py:369: SyntaxWarning: invalid escape sequence '\:'
  Given a query, replaces all occurrences of 'example'[x:y] with 'example'[x\:y].


Loading configurations from /app/pyproject.toml.

Settings changed:

Config,value
autolimit,False
autopolars,True
feedback,1
named_parameters,enabled


In [31]:
df = (
    conn.sql('''
    SELECT *
    FROM ph_mte25__party_raw
    LIMIT 10
    ''')
    .pl()
)
df

name,vote_count,party,region,batch,timestamp
str,i64,str,str,i32,"datetime[μs, Etc/UTC]"
"""4PS""",31892,"""4PS|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC
"""PPP""",7243,"""PPP|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC
"""FPJ PANDAY BAYANIHAN""",7406,"""FPJ PANDAY BAYANIHAN|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC
"""KABATAAN""",2483,"""KABATAAN|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC
"""DUTERTE YOUTH""",64928,"""DUTERTE YOUTH|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC
"""ML""",3049,"""ML|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC
"""PBBM""",1501,"""PBBM|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC
"""P3PWD""",1993,"""P3PWD|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC
"""MURANG KURYENTE""",1059,"""MURANG KURYENTE|GROUP""","""BARMM""",117,2025-05-15 06:41:56 UTC


In [32]:
df = st.read_file('./data/phl_adm_psa_namria_20231106_shp/phl_admbnda_adm1_psa_namria_20231106.shp')
df

ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,Shape_Leng,Shape_Area,ADM1ALT1EN,AREA_SQKM,geometry
str,str,str,str,date,date,date,f64,f64,str,f64,binary
"""Region I (Ilocos Region)""","""PH01""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,14.99505,1.043983,"""Ilocos Region""",12307.350828,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00\x7f\x00\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x008\xb1\x02\x00\x90\xf7y\x12,>^@`\xa4]\x1ay\x942@\x88\xc1\x1b#0>^@\x80\xbaO\xa2f\x942@\x08Y""…"
"""Region II (Cagayan Valley)""","""PH02""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,19.139048,2.241812,"""Cagayan Valley""",26387.729555,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00Y\x00\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x0e\x00\x00\x100%\xc5\xb9|^@\xd04'\xbd\\x1a5@@\x15\xbc]\xb9|^@\xd04'\xbd\\x1a5@8\xea""…"
"""Region III (Central Luzon)""","""PH03""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,15.949563,1.793513,"""Central Luzon""",21304.155519,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00\x11\x00\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x00\xe5?\x04\x00h\xcd\xaa\xca\xfc\x8e^@\x80[5\xb1\xbb|0@\xf0)\xf6\xc0\xfc\x8e^@\x90\x9e\xa3t\xba|0@\x10b""…"
"""Region IV-A (Calabarzon)""","""PH04""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,27.625494,1.32672,"""Calabarzon""",15846.630559,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00r\x00\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x00\xe4\xf3\x06\x00\xd8\x11\xbd\x15\xb4\x93^@\xe0:\x01R=4,@@\x12w\xdb\xb6\x93^@\x20\xa4\x87\xde83,@\x88\xe7""…"
"""Region V (Bicol Region)""","""PH05""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,44.923243,1.446324,"""Bicol Region""",17338.375533,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00\x8a\x01\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x00q\x01\x00\x00\x00D\xad\x15?\xbf^@\x80\xad\xbd\x96*v'@@\x8b\x16T?\xbf^@\x20\xa8\x1a\xde(v'@\xa0\x95""…"
…,…,…,…,…,…,…,…,…,…,…,…
"""National Capital Region (NCR)""","""PH13""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,2.303584,0.050207,"""NCR""",598.539573,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\xa4e\x00\x00\x98\xaf\xacP^F^@\xc0\x0c\xd9\xd6\xd5\x89-@\x20\x14\xc8J^F^@\xc0\x88\xe7\x07\xd6\x89-@\xe8\xc37=]F^@\xa0\xefc""…"
"""Cordillera Administrative Regi…","""PH14""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,8.027454,1.546712,"""CAR""",18192.644052,"b""\x01\x03\x00\x00\x20\xe6\x10\x00\x00\x01\x00\x00\x00\x0d\x10\x00\x00X\xa6\xc8\xbc61^@\x90RpU\xb220@\x08\xecE\xe251^@\x90U\xc2n\xb320@\xb0n\xdd\xb6w0^@\x80:""""…"
"""Region XIII (Caraga)""","""PH16""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,28.166743,1.54082,"""Caraga""",18747.165762,"b""\x01\x06\x00\x00\x20\xe6\x10\x00\x00\x1c\x01\x00\x00\x01\x03\x00\x00\x00\x01\x00\x00\x00\x16\x00\x00\x00\xf06\x1b\x0a\xe3\x95_@\x20\x09\xacw\x13\xc1!@\xb8\x87Y\xdc\xdd\x95_@\x80\xe8\xde\xd7\x09\xc1!@\xf8\x99""…"


In [36]:
df.st.to_wkt()

ADM1_EN,ADM1_PCODE,ADM0_EN,ADM0_PCODE,date,validOn,validTo,Shape_Leng,Shape_Area,ADM1ALT1EN,AREA_SQKM,geometry
str,str,str,str,date,date,date,f64,f64,str,f64,str
"""Region I (Ilocos Region)""","""PH01""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,14.99505,1.043983,"""Ilocos Region""",12307.350828,"""MULTIPOLYGON (((120.97144 18.5…"
"""Region II (Cagayan Valley)""","""PH02""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,19.139048,2.241812,"""Cagayan Valley""",26387.729555,"""MULTIPOLYGON (((121.948839 21.…"
"""Region III (Central Luzon)""","""PH03""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,15.949563,1.793513,"""Central Luzon""",21304.155519,"""MULTIPOLYGON (((122.234179 16.…"
"""Region IV-A (Calabarzon)""","""PH04""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,27.625494,1.32672,"""Calabarzon""",15846.630559,"""MULTIPOLYGON (((122.307867 14.…"
"""Region V (Bicol Region)""","""PH05""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,44.923243,1.446324,"""Bicol Region""",17338.375533,"""MULTIPOLYGON (((122.988225 11.…"
…,…,…,…,…,…,…,…,…,…,…,…
"""National Capital Region (NCR)""","""PH13""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,2.303584,0.050207,"""NCR""",598.539573,"""POLYGON ((121.099507 14.76921,…"
"""Cordillera Administrative Regi…","""PH14""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,8.027454,1.546712,"""CAR""",18192.644052,"""POLYGON ((120.768966 16.198034…"
"""Region XIII (Caraga)""","""PH16""","""Philippines (the)""","""PH""",2022-11-09,2023-11-06,null,28.166743,1.54082,"""Caraga""",18747.165762,"""MULTIPOLYGON (((126.341982 8.8…"
